# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [ ]:
import json
with open('addres-book.json', 'r') as f:
  reader = json.load(f)
  for i in reader:
    print(i['email'])

faina@mail.ru
robert@mail.ru


2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [ ]:
for i in reader:
  print(*i['phones'])

{'phone': '232-19-55'} {'phone': '+7 (916) 232-19-55'}
{'phone': '111-19-55'} {'phone': '+7 (916) 445-19-55'}


3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [ ]:
from bs4 import BeautifulSoup
with open('addres-book-q.xml', 'r') as f1:
  reader_xml = BeautifulSoup(f1, 'xml')
  
list_of_names = []
list_of_phones = []
for ad in reader_xml.address_book.find_all('address'):
  list_of_names += [person.next for person in ad.find_all('name')]
  list_of_phones.append([phone.next for phone in ad.phones.find_all('phone')])

dict_name_phone = {i:j for i, j in zip(list_of_names,list_of_phones)}
for i in dict_name_phone:
  print(i,':', dict_name_phone[i])

Aicha Barki : ['+ (213) 6150 4015', '+ (213) 2173 5247']
Francisco Domingos : ['+ (244-2) 325 023', '+ (244-2) 325 023']
Maria Luisa : ['+ (244) 4232 2836']
Abraao Chanda : ['+ (244-2) 325 023', '+ (244-2) 325 023']
Beatriz Busaniche : ['+ (54-11) 4784 1159']
Francesca Beddie : ['+ (61-2) 6274 9500', '+ (61-2) 6274 9513']
Graham John Smith : ['+ (61-3) 9807 4702']


## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
import json
with open('contributors_sample.json', 'r') as f:
  reader = json.load(f)
  for i in reader[:3]:
    print(i)

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
mails = [i['mail'] for i in reader]
domen = set(i[i.index('@'):] for i in mails)
print(domen)

{'@yahoo.com', '@hotmail.com', '@gmail.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def search(username):
  for i in reader:
    if i['username'] == username:
      print(i)
    else:
      raise ValueError('Нет такого пользователя')
search('sheilaadam')

ValueError: ignored

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
female, male = 0, 0 
for i in reader:
  if i['sex'] == 'F':
    female += 1
  elif i['sex'] == 'M':
    male += 1
print(f'female = {female}\nmale = {male}')

female = 2136
male = 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
import pandas as pd
contributors = pd.DataFrame.from_dict(reader)[['id', 'username','sex']]
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
merge_recipes = recipes.merge(contributors, on = 'id',how = 'left')
merge_recipes[['username']].isnull().sum() 

username    29830
dtype: int64

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
user_jobs = {i['username']:i['jobs'] for i in reader}
jobs = set(job for list_jobs in [i['jobs'] for i in reader] for job in list_jobs)
dict_of_jobs = dict.fromkeys(jobs)
for user in user_jobs:
  for job in user_jobs[user]:
    if dict_of_jobs[job] == None: dict_of_jobs[job] = [user]
    else: dict_of_jobs[job].append(user)
for h in dict_of_jobs:
  print(h, dict_of_jobs[h])

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
import pickle, json
with open('job_people.pickle', 'wb') as f:
  pickle.dump(dict_of_jobs, f)
with open('job_people.json', 'w') as f2:
  json.dump(dict_of_jobs, f2, indent = 4)



In [ ]:
import os.path
print(os.path.getsize('job_people.pickle'))
print(os.path.getsize('job_people.json'))

129607
381248


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as f3:
  reader = pickle.load(f3)
  print(reader)
  print(type(reader))

{'Embryologist, clinical': ['jessecowan', 'bridgetjones', 'alexanderproctor', 'cheryl29', 'jillwilliams', 'ihill', 'leslie58', 'boonerachel', 'fhughes', 'wgaines', 'jasonestes', 'npadilla', 'jimmy12', 'rrussell', 'jeremiaharmstrong', 'mark73', 'brandigreen', 'andrewlee', 'urangel', 'trevinorachel'], 'Conservation officer, nature': ['pphillips', 'kjenkins', 'kathryngonzales', 'scott92', 'johnsonsavannah', 'michele56', 'jodigonzalez', 'brandon60', 'stephanie81', 'nataliejennings', 'tammybailey', 'jennifer45', 'hernandezlaura', 'frederickrodgers', 'jeremymontoya', 'wmyers', 'melissa94', 'gmurphy', 'bbutler'], 'Psychotherapist': ['huntterri', 'huntterri', 'teresa71', 'gregorygoodman', 'gsmith', 'dylan44', 'seanbarnett', 'seanbarnett', 'lgreen', 'evansjason', 'zeverett', 'anthony22', 'rogerssharon', 'mackenziehernandez', 'blacknicholas', 'byrdsally', 'andersongerald', 'kvance', 'michaeljohnson', 'gary46', 'willismartin', 'brendahill'], 'Secondary school teacher': ['martindonald', 'btownsend

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
from bs4 import BeautifulSoup
with open('steps_sample.xml', 'r') as f:
  reader = BeautifulSoup(f, 'xml')


In [ ]:
import json

In [ ]:
ides = []
steps = []
for recipe in reader.recipes.find_all('recipe'):
  ides += [id.next for id in recipe.find_all('id')]
  steps.append([step.next for step in recipe.steps.find_all('step')])
dict_id_steps = {i:j for i,j in zip(ides, steps)}
with open('steps_sample.json', 'w') as f:
  json.dump(dict_id_steps, f, indent = 4)


3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
steps_len = []
dict_nsteps_id = {}
for recipe in reader.recipes.find_all('recipe'):
  num_steps = len([step.next for step in recipe.steps.find_all('step')])
  if num_steps not in dict_nsteps_id:
    dict_nsteps_id[num_steps] = [id.next for id in recipe.find_all('id')]
  else:
    dict_nsteps_id[num_steps] += [id.next for id in recipe.find_all('id')]
for i in dict_nsteps_id:
  print(i, ':', dict_nsteps_id[i])

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
id_recipe = []
for recipe in reader.find_all('recipe'):
  for steps in recipe.find_all('steps'):
    if 'has_hours' in steps.step.attrs or 'has_minutes' in steps.step.attrs:
      id_recipe.append(recipe.find('id').next)
print(id_recipe)

['44123', '33246', '428566', '180817', '482111', '292147', '53353', '140173', '289025', '101260', '385415', '3441', '21629', '151131', '58045', '407621', '139718', '103153', '56202', '392927', '13709', '234169', '373269', '229599', '184074', '141308', '143286', '336314', '307870', '76727', '181765', '50337', '159732', '19104', '339114', '143795', '476489', '136514', '402687', '291364', '66815', '23964', '499403', '395585', '39398', '384056', '97140', '238257', '512016', '335405', '192298', '260930', '116397', '77581', '108473', '98558', '32515', '202823', '368003', '24447', '457379', '69539', '133919', '386956', '183494', '492732', '11040', '179633', '431088', '410251', '187969', '13225', '289580', '184531', '176060', '474905', '235297', '86386', '117668', '39412', '259813', '126810', '119049', '6842', '163569', '188405', '290595', '159899', '119601', '212473', '2866', '98464', '121225', '386025', '66559', '111877', '207753', '55114', '369435', '160705', '515586', '392122', '317893', '

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
#из предыдущего задания
from bs4 import BeautifulSoup
import pandas as pd
with open('steps_sample.xml', 'r') as f:
  reader = BeautifulSoup(f, 'xml')

In [ ]:
#создали словарь вида id:n_steps
list_nsteps = {}
for recipe in reader.recipes.find_all('recipe'):
  num_steps = len([step.next for step in recipe.steps.find_all('step')])
  list_nsteps[int(recipe.find('id').next)]=num_steps
#создаём таблицу на основе словаря
table = pd.DataFrame(list_nsteps.items(), columns=['id', 'n_steps'])

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipes['n_steps'] = recipes['n_steps'].fillna(table['n_steps']) 
recipes

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
print(recipes['n_steps'].isnull().sum())
recipes['n_steps'] = recipes['n_steps'].astype('int')
recipes.to_csv('recipes_sample_with_filled_nsteps.csv')

0
